In [7]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

## 앙상블을 해보자!
- k fold 용으로 각 fold 불러오는 것 구현

In [9]:
# config file 들고오기
cfg = Config.fromfile('./configs/custom_augmentation/faster_rcnn_mosaic.py')

# Multi scale
cfg.data.test.pipeline[1]['img_scale'] = [(1000, 600),(1333, 800),(1666, 1000)]

# checkpoint path
epoch = 24
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

print(f'Config:\n{cfg.pretty_text}')

# checkpoint path
checkpoint_path0 = './work_dirs/faster_rcnn_kfold0/epoch_12.pth'
checkpoint_path1 = './work_dirs/faster_rcnn_kfold1/epoch_12.pth'
checkpoint_path2 = './work_dirs/faster_rcnn_kfold2/epoch_12.pth'
checkpoint_path3 = './work_dirs/faster_rcnn_kfold3/epoch_12.pth'
checkpoint_path4 = './work_dirs/faster_rcnn_kfold4/epoch_12.pth'

In [80]:
# 데이터셋은 한번만 해도 될듯
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [81]:
model0 = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint0 = load_checkpoint(model0, checkpoint_path0, map_location='cpu')

model0.CLASSES = dataset.CLASSES
model0 = MMDataParallel(model0.cuda(), device_ids=[0])

2021-05-12 00:54:46,226 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-12 00:54:46,227 - mmdet - INFO - Use load_from_torchvision loader
2021-05-12 00:54:46,452 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader


In [82]:
model1 = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint1 = load_checkpoint(model1, checkpoint_path1, map_location='cpu')

model1.CLASSES = dataset.CLASSES
model1 = MMDataParallel(model1.cuda(), device_ids=[0])

2021-05-12 00:54:47,202 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-12 00:54:47,203 - mmdet - INFO - Use load_from_torchvision loader
2021-05-12 00:54:47,391 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader


In [83]:
model2 = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint2 = load_checkpoint(model2, checkpoint_path2, map_location='cpu')

model2.CLASSES = dataset.CLASSES
model2 = MMDataParallel(model2.cuda(), device_ids=[0])

2021-05-12 00:54:48,114 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-12 00:54:48,115 - mmdet - INFO - Use load_from_torchvision loader
2021-05-12 00:54:48,289 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader


In [84]:
model3 = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint3 = load_checkpoint(model3, checkpoint_path3, map_location='cpu')

model3.CLASSES = dataset.CLASSES
model3 = MMDataParallel(model3.cuda(), device_ids=[0])

2021-05-12 00:54:48,982 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-12 00:54:48,983 - mmdet - INFO - Use load_from_torchvision loader
2021-05-12 00:54:49,212 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader


In [85]:
model4 = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint4 = load_checkpoint(model4, checkpoint_path4, map_location='cpu')

model4.CLASSES = dataset.CLASSES
model4 = MMDataParallel(model4.cuda(), device_ids=[0])

2021-05-12 00:54:49,914 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-12 00:54:49,915 - mmdet - INFO - Use load_from_torchvision loader
2021-05-12 00:54:50,093 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader


In [86]:
output0 = single_gpu_test(model0, data_loader, show_score_thr=0.05)
output1 = single_gpu_test(model0, data_loader, show_score_thr=0.05)
output2 = single_gpu_test(model2, data_loader, show_score_thr=0.05)
output3 = single_gpu_test(model0, data_loader, show_score_thr=0.05)
output4 = single_gpu_test(model0, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 34.0 task/s, elapsed: 25s, ETA:     0s[                                                  ] 0/837, elapsed: 0s, ETA:[                                                  ] 0/837, elapsed: 0s, ETA:[                                                  ] 0/837, elapsed: 0s, ETA:[                                                  ] 0/837, elapsed: 0s, ETA:

In [62]:
!pip install ensemble-boxes

In [2]:
import mmcv

## 다른 모델 앙상블
- 저장된 pkl 파일 불러오기
- kfold 앙상블시 위에서 inference한 output0~4까지 이용

In [31]:
output0 = mmcv.load('./ensemble_pkl/universenet.pkl')
output1 = mmcv.load('./ensemble_pkl/vfnet_5601.pkl')
output2 = mmcv.load('./ensemble_pkl/gflv2.pkl')
output3 = mmcv.load('./ensemble_pkl/swin.pkl')
# output4 = mmcv.load('./ensemble_pkl/')
# output5 = mmcv.load('./ensemble_pkl/')

In [5]:
from ensemble_boxes import *

In [42]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()
iou_thr = 0.55
skip_box_thr = 0.0
sigma = 0.1

class_num = 11
for i in range(len(output0)):
    out0 = output0[i]
    out1 = output1[i]
    out2 = output2[i]
#     out3 = output3[i][0]
#     out4 = output4[i]
#     out5 = output5[i]
    weights = [0.8,
               0.8,
               0.8,
#                0.15, 
#                0.5
#                0.5
              ]
    
    boxes_list0 = []
    boxes_list1 = []
    boxes_list2 = []
#     boxes_list3 = []
#     boxes_list4 = []
#     boxes_list5 = []

    scores_list0 = []
    scores_list1 = []
    scores_list2 = []
#     scores_list3 = []
#     scores_list4 = []
#     scores_list5 = []

    labels_list0 = []
    labels_list1 = []
    labels_list2 = []
#     labels_list3 = []
#     labels_list4 = []
    for j in range(class_num):
        boxes_list0 += (out0[j][:, :4]/512).tolist()
        boxes_list1 += (out1[j][:, :4]/512).tolist()
        boxes_list2 += (out2[j][:, :4]/512).tolist()
#         boxes_list3 += (out3[j][:, :4]/512).tolist()
#         boxes_list4 += (out4[j][:, :4]/512).tolist()

        scores_list0 += out0[j][:, 4].tolist()
        scores_list1 += out1[j][:, 4].tolist()
        scores_list2 += out2[j][:, 4].tolist()
#         scores_list3 += out3[j][:, 4].tolist()
#         scores_list4 += out4[j][:, 4].tolist()
        
        # 0 class는 분류 못해서 +1
        labels_list0 += [j] * len(out0[j][:, 4].tolist())
        labels_list1 += [j] * len(out1[j][:, 4].tolist())
        labels_list2 += [j] * len(out2[j][:, 4].tolist())
#         labels_list3 += [j] * len(out3[j][:, 4].tolist())
#         labels_list4 += [j] * len(out4[j][:, 4].tolist())

    
    boxes_list = [boxes_list0, 
                  boxes_list1, 
                  boxes_list2, 
#                   boxes_list3, 
#                   boxes_list4
                 ]
    scores_list = [scores_list0, 
                   scores_list1, 
                   scores_list2, 
#                    scores_list3, 
#                    scores_list4
                  ]
    labels_list = [labels_list0, 
                   labels_list1, 
                   labels_list2, 
#                    labels_list3, 
#                    labels_list4
                  ]

    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    
    predict_zip = list(zip(boxes, scores, labels))
    predict_zip.sort(key = lambda x : (x[2], -x[1]))
    for box, score, label in predict_zip:
        if score>=0.1:
            prediction_string += str(int(label)) + ' ' + f'{score:.8f}' + ' ' + f'{box[0]*512:.5f}' + ' ' + f'{box[1]*512:.5f}' + ' ' + f'{box[2]*512:.5f}' + ' ' + f'{box[3]*512:.5f}' + ' '
        else:
            prediction_string += str(int(label)) + ' ' + f'{score:.9f}' + ' ' + f'{box[0]*512:.5f}' + ' ' + f'{box[1]*512:.5f}' + ' ' + f'{box[2]*512:.5f}' + ' ' + f'{box[3]*512:.5f}' + ' '

    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])

epoch = 12
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}_ensemble3(08 08 08)iou0565_final.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.056775387 159.57796 67.95561 255.01704 167...,batch_01_vt/0021.jpg
1,0 0.098784536 286.33960 337.71945 306.88809 37...,batch_01_vt/0028.jpg
2,0 0.11180874 235.40033 460.90344 302.91647 511...,batch_01_vt/0031.jpg
3,0 0.088503465 293.09857 156.78738 365.58768 22...,batch_01_vt/0032.jpg
4,0 0.091454516 37.98398 459.40359 88.38715 473....,batch_01_vt/0070.jpg
